In [1]:
import docx2txt
import pandas as pd

### setup
filePath = 'engBook_statements.docx'
text = docx2txt.process(filePath)
texts = []
introduction_text = ""

amount_texts = 84
amount_questions = 10
tmpABC = ["A)", "B)", "C)", "D)", "E)"]
amount_choices = len(tmpABC)

#split document into text+question+answer blocks (50 texts-> 50 blocks)
for i in range(0,amount_texts, amount_questions):
    tmp=text.split("Statement " + str(i+1), 1)
    tmp[0]=tmp[0].replace('\n', ' ').replace('\r', '')
    texts.append("Statement " + str(i-amount_questions+1)  + tmp[0])
    #print("Question " + str(i-amount_questions+1) + " " + tmp[0])
    if(len(tmp) > 1):
        text=tmp[1]
    
introduction_text=texts.pop(0).replace("Text 0 ", "")

print(texts)


['Statement 1:  "The company\'s policy on remote work aims to improve work-life balance for its employees."  A) The policy is designed to make it harder for employees to manage their personal and professional lives. (True/False) False   B) Improving employees\' satisfaction by balancing their work and personal life is a goal of the policy. (True/False) True   C) The company does not believe work-life balance is important for employees who work remotely. (True/False) False   D) The policy specifically targets reducing the amount of work employees have to do at home. (True/False) False   E) One of the intentions of the policy is to ensure employees have adequate time for personal activities. (True/False) True    Statement 2:  "The primary objective of the new educational program is to enhance students\' critical thinking skills."  A) The new program is mainly focused on improving students\' ability to analyze and evaluate information critically. (True/False) True   B) A significant goal 

In [2]:
#split each block into QuestionType, QuestionText, Choice1 - Choice7
#choice which is "true" is marked with a "*" before the text

#### setup
tmpTexts=[]
tmpTexts.append(texts[0])
tmpTexts.append(texts[1])

for main_index,block in enumerate(tmpTexts):
    # setup
    choices=[]
    question_text = ""
    questions = []
    excel_rows = []
    dataFrame_columns = ["QuestionType", "QuestionText", "Explanation", "Choice1", "Choice2", "Choice3", "Choice4", "Choice5", "Choice6", "Choice7", "Choice8", "Choice9", "Choice10"]
    writer = pd.ExcelWriter( "Statements " + str(main_index+1) + ".xlsx", engine="xlsxwriter")
    tmpQuestions=block
    #print(main_index)
    #print(tmpQuestions)
    
    # Questions Blöcke aufteilen (question + choices)
    for i in range(amount_questions):
        if(i == amount_questions-1):
            tmp = tmpQuestions.split("Statement " + str(main_index*10 + i+2), 1)
            #print(tmp)
            questions.append(tmp[0])
            #questions.append("Question " + str(main_index*10 + i+2) + tmp[1])
            
        else:
            tmp = tmpQuestions.split("Statement " + str(main_index*10+i+2), 1)
            #print(tmp)
            questions.append(tmp[0])
            tmpQuestions = "Statement " + str(main_index*10+i+2) + tmp[1]
    #print(questions)  
    # choices aufteilen & true/false bestimmen
    for index,question in enumerate(questions):
        choices=[]
        explanation="<strong>Correct answers: </strong>"
        tmp = question.split(tmpABC[0], 1)
        question_text = "<strong>" + tmp[0].replace("Statement " + str(main_index*10+index+1)+ ":", "") + "</strong>"
        tmpChoices=tmpABC[0] + tmp[1]
        for index, letter in enumerate(tmpABC):
            if(letter == tmpABC[0]):
                continue
            elif(letter == tmpABC[-1]):
                tmp = tmpChoices.split("  " + letter, 1)
                #print(tmp)
                choices.append(tmp[0].replace(tmpABC[-2] + " ",""))
                choices.append(tmp[1].replace(" ", "", 1))
            else:
                tmp = tmpChoices.split("  " + letter, 1)
                #print(tmp)
                choices.append(tmp[0].replace(tmpABC[index-1] + " ", ""))
                tmpChoices=letter + tmp[1]
        #print(choices)
        for index,choice in enumerate(choices):
            tmp=choice.split(" (True/False) ")
            if(tmp[1].replace(" ", "") == "True"):
                choices[index] = "*" + tmp[0]
                explanation = explanation + "<br>" + '<span style="color: rgb(97, 189, 109);">' +  tmpABC[index] + " " + tmp[0] + '</span>'+ " "
            else:
                choices[index] = tmp[0]
                explanation = explanation + "<br>" + '<span style="color: rgb(226, 80, 65);">' +  tmpABC[index] + " " + tmp[0] + '</span>'+ " "
        
        ### pandas setup (writing to excel table)
        values=["MA", question_text, explanation] + choices + ["","","","",""]
        excel_rows.append(values)

    df = pd.DataFrame ( excel_rows, columns = dataFrame_columns)
    df.to_excel(writer, sheet_name="QUESTIONS", startrow=0, header=True, index=False)
    writer.close()          
    # print(question_text)
    # print(tmpQuestionText)
    # print(choices)
    # print(questions)